In [1]:
import pandas as pd
import plotly.express as px

In [2]:
card_01 = pd.read_csv("df_card01.csv")
card_02 = pd.read_csv("df_card02.csv")
card_03 = pd.read_csv("df_card03.csv")
card_04 = pd.read_csv("df_card04.csv")
card_05 = pd.read_csv("df_card05.csv")
card_06 = pd.read_csv("df_card06.csv")

In [3]:
card = pd.concat([card_01, card_02, card_03, card_04, card_05, card_06], ignore_index= True)
card.head()

,Unnamed: 0,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,0,20200104,11740,성내3동,70,약국,463.0,5843230.0
1,1,20200104,11740,성내3동,70,치과의원,33.0,7835550.0
2,2,20200104,11740,성내3동,70,한의원,53.0,4589800.0
3,3,20200104,11740,성내3동,70,의원,339.0,9267240.0
4,4,20200104,11740,성내3동,61,자동차정비,19.0,4441000.0


In [4]:
#20200101->2020-01-01
card['receipt_dttm']=card['receipt_dttm'].astype(str)
card['receipt_dttm']=card['receipt_dttm'].apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
#datetime으로
card['receipt_dttm']=pd.to_datetime(card['receipt_dttm'])
#월, 주차, 표시
card['주차']=card['receipt_dttm'].dt.weekofyear
card['월']=card['receipt_dttm'].dt.month
card.head()

,Unnamed: 0,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,주차,월
0,0,2020-01-04,11740,성내3동,70,약국,463.0,5843230.0,1,1
1,1,2020-01-04,11740,성내3동,70,치과의원,33.0,7835550.0,1,1
2,2,2020-01-04,11740,성내3동,70,한의원,53.0,4589800.0,1,1
3,3,2020-01-04,11740,성내3동,70,의원,339.0,9267240.0,1,1
4,4,2020-01-04,11740,성내3동,61,자동차정비,19.0,4441000.0,1,1


In [8]:
induty_nm = pd.read_csv("품목명.csv")
induty_nm.head()

,num,nm
0,10,숙박
1,11,교통수단
2,20,여가 용품
3,21,여가 장소
4,22,문화


### 함수 코드 완성

In [28]:
def data_mean(card_name, ads,name):

    df=card_name[card_name['adstrd_code']==ads]
    
    is_week=df['주차']<=9
    card_before=df[is_week]
    card_after=df.drop(df[is_week].index)
    
    #1,2월 평균값 계산
    card_before=card_before.groupby(['mrhst_induty_cl_code'],as_index=False).selng_cascnt.mean()
    
    #품목별 평균값을 dict형태로 지정
    card_mean=dict()
    for i in range(len(card_before)):
        card_mean[card_before['mrhst_induty_cl_code'][i]]=card_before['selng_cascnt'][i]
    
    #3월부터 상대지수 계산(2020년 9주차 부터)
    card_a=card_after.groupby(['주차','mrhst_induty_cl_code'],as_index=False).selng_cascnt.mean()
    
    #상대지수 열로 생성
    add_col=[]
    for i in range(len(card_a)):
        mean=card_mean[card_a['mrhst_induty_cl_code'][i]]
        add=(card_a['selng_cascnt'][i]-mean)/mean
        add_col.append(add)
    card_a['corr']=add_col
    
    #품목명 지정
    induty=dict()
    for i in range(len(induty_nm)):
        induty[induty_nm['num'][i]]=induty_nm['nm'][i]

    add_col=[]
    for i in range(len(card_a)):
        add_col.append(induty[card_a['mrhst_induty_cl_code'][i]])
    card_a['induty_nm']=add_col
    
    #plot
    fig = px.bar(card_a, x="induty_nm", y="corr",color="induty_nm",
                 animation_frame="주차", animation_group="induty_nm", range_y=[-5,500])

    fig.update_layout(
        title={
            'text': "품목별 변화 추이-"+name,
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    fig.show()
    
    return

#### 상대값 계산

In [30]:
# 용산구 상대값 : 11170
data_mean(card, 11170,'서울시 용산구')


In [31]:
# 서초구 상대값 : 11650
data_mean(card, 11650,"서울시 서초구")

In [32]:
# 관악구 상대값 : 11620
data_mean(card, 11620,"서울시 관악구")

In [33]:
# 강동구 상대값 : 11740
data_mean(card, 11740,"서울시 강동구")